In [1]:
from glob import glob
import numpy as np

list(set([x.split("/")[-1].split("_")[0] for x in glob("/home/shayaan/Desktop/aimet/my_mmdet/pred_results/det_infer_raw_results/*.npy")]))

['179487']

In [7]:
img_id = 179487
corr_raw = {x.split("/")[-1].split(".")[0]: np.load(x) for x in glob(f"/home/shayaan/Desktop/aimet/temp/mmdetection/corr_waale/{img_id}*.npy")}
my_raw = {x.split("/")[-1].split(".")[0]: np.load(x) for x in glob(f"/home/shayaan/Desktop/aimet/my_mmdet/pred_results/det_infer_raw_results/{img_id}*.npy")}

len(corr_raw), len(my_raw)

(6, 6)

In [8]:
# function to calculate avg mse of 2 numpy arrays of same sizes
def mse(arr1, arr2):
    return np.mean((arr1 - arr2)**2)

corr_raw = sorted(corr_raw.items(), key=lambda x: x[0])
print(img_id)
for k, v in corr_raw:
    other_v = my_raw[k]
    print(mse(v, other_v), "<----", v.shape)
    print()

179487
0.0 <---- (1, 4, 20, 20)

0.0 <---- (1, 4, 40, 40)

0.0 <---- (1, 4, 80, 80)

0.0 <---- (1, 80, 20, 20)

0.0 <---- (1, 80, 40, 40)

0.0 <---- (1, 80, 80, 80)

